# Algoritmos Genéticos

    ▪Ramo dos algoritmos evolucionários
    ▪Técnica de busca baseada na metáfora do processo biológico de evolução natural.
    ▪Populações de indivíduos são criadas e submetidas aos operadores genéticos: 
        ▪Seleção;
        ▪Recombinação (crossover);
        ▪Mutação;
        ▪Elitismo.

## Minimizando a equação X^2 + Y^2

## Bitset

In [1]:
import numpy as np
import random

class BitSet:
    def __init__(self, size):
        self.bits = np.full((1, size), False)

    def get(self, index):
        return self.bits[0, index]

    def set(self, index, value):
        self.bits[0, index] = not value

    def flip(self, index):
        self.bits[0, index] = not self.bits[0, index]

    def debug(self):
        print(self.bits)
        
        

In [2]:
b = BitSet(5)
b.flip(2)
b.debug()

[[False False  True False False]]


## Cromossomo

In [3]:
class Chromossome:
    def __init__(self, x = None, y = None):
        if x == None:
            x = random.randint(-15, 15)

        if y == None:
            y = random.randint(-15, 15)

        self.__genes = Chromossome.get_genotype(x, y)

    def get_genes(self):
        return self.__genes

    def set_genes(self, genes):
        self.__genes = genes

    def to_string(self):
        chr_str = "G = ["

        for i in range(10):
            chr_str += (i == 5 and " " or "") + (self.__genes.get(i) and "1" or "0")

        x, y = Chromossome.get_fenotype(self.__genes)

        chr_str += "], F = [" + str(x) + ", " + str(y) + "]"

        return chr_str

    @staticmethod
    def get_genotype(x, y):
        bits = BitSet(10)

        xy_binary = "{:05b}".format(x) + "{:05b}".format(y)

        for i in range(10):
            bits.set(i, xy_binary[i] == '0')

        return bits

    @staticmethod
    def get_fenotype(genes):

            # xxxxx yyyyy (genótipo)
            # 01234 56789 (indices)
            # S8421 S8421 (s == 0 -> Positivo; s == 1 -> Negativo)
            # Ex : 11000 = -8        01000 = 8        11010 = -10         01010 = 10

        x = (8 * genes.get(1) +
         4 * genes.get(2) +
         2 * genes.get(3) +
         1 * genes.get(4))
    
        if genes.get(0) == 1:
            x *= -1

        y = (8 * genes.get(6) +
            4 * genes.get(7) +
            2 * genes.get(8) +
            1 * genes.get(9))
    
        if genes.get(5) == 1:
            y *= -1
            
        return x, y

In [4]:
c = Chromossome ()
c.to_string()

'G = [01101 01010], F = [13, 10]'

## Problema

In [5]:
class Problem:

    @staticmethod
    def f(x, y):
        return x**2 + y**2

    @staticmethod
    def f_chromossome(chromossome):   # verificando o resultado no fenotico
        genes = chromossome.get_genes()   # pegando o genotico ( binario )
        x, y = Chromossome.get_fenotype(genes) # com o binario ( gene ) pegando o fenotipo ( decimal )
        return Problem.f(x, y)

    @staticmethod
    def g(x, y):    # o maior eh o melhor cromossomo
        return 1 / (1 + Problem.f(x, y))

    @staticmethod
    def g_chromossome(chromossome):   # usado para achar o melhor cromossomo ( o maior valor que eh o melhor )
        genes = chromossome.get_genes()
        x, y = Chromossome.get_fenotype(genes)
        return Problem.g(x, y)

    @staticmethod
    def f_average(population):   # a media do score f
        avg = 0
        for chromossome in population:
            avg += Problem.f_chromossome(chromossome)
        avg /= len(population)
        return avg

    @staticmethod
    def g_average(population):  # a media do score g ( media score da populacao )
        avg = 0
        for chromossome in population:
            avg += Problem.g_chromossome(chromossome)
        avg /= len(population)
        return avg

## Utils

In [6]:
class GeneticUtils:

    @staticmethod
    def find_best_chromossome(population):
        best_chromossome = None

        for chromossome in population:
            score = Problem.g_chromossome(chromossome)

            if best_chromossome is None or score > Problem.g_chromossome(best_chromossome):
                best_chromossome = chromossome

        return best_chromossome

    @staticmethod
    def find_worst_chromossome(population):
        worst_chromossome = None

        for chromossome in population:
            score = Problem.g_chromossome(chromossome)

            if worst_chromossome is None or score < Problem.g_chromossome(worst_chromossome):
                worst_chromossome = chromossome

        return worst_chromossome

    @staticmethod
    def format_chromossome(chromossome):
        return f"{chromossome.to_string()}, Score = {Problem.g_chromossome(chromossome):.3f}"

In [7]:
p = [Chromossome() for _ in range(5)]
print("\n".join([c.to_string() for c in p]))

G = [11010 01100], F = [-10, 12]
G = [10110 00000], F = [-6, 0]
G = [01011 01111], F = [11, 15]
G = [01111 01011], F = [15, 11]
G = [01110 10010], F = [14, -2]


In [8]:
b = GeneticUtils.find_best_chromossome(p)
w = GeneticUtils.find_worst_chromossome(p)

print( GeneticUtils.format_chromossome(b))
print( GeneticUtils.format_chromossome(w))

G = [10110 00000], F = [-6, 0], Score = 0.027
G = [01011 01111], F = [11, 15], Score = 0.003


## Genetic Operations

In [9]:
class GeneticOperators:

    MUTATION_PROBABILITY = 0.05   # probabilidade de acontecer alguma mutacao

    @staticmethod
    def selection(population):      # seleciona dois pais aleatorios
        parent1 = random.choice(population)
        parent2 = random.choice(population)

        while parent1 is parent2:               # verifica se sao iguais, gera ate serem diferentes
            parent1 = random.choice(population)
            parent2 = random.choice(population)

        print(f"1st parent chosen for crossover: {GeneticUtils.format_chromossome(parent1)}")
        print(f"2nd parent chosen for crossover: {GeneticUtils.format_chromossome(parent2)}")

        return parent1, parent2

    @staticmethod
    def crossover(population, parent1, parent2):      # adicionando dois filhos para a populacao
        crossover_point = random.randint(1, 9)
        print(f"Crossover will happen at point {crossover_point}")

        parent1_genes = parent1.get_genes()
        parent2_genes = parent2.get_genes()

        child1_genes = BitSet(10)
        child2_genes = BitSet(10)

        for i in range(0, crossover_point):     # crossover antes do corte   (0, 5)
            child1_genes.set(i, not parent1_genes.get(i)) # filho1 = (0,5) -> pai1
            child2_genes.set(i, not parent2_genes.get(i)) # filho2 = (0,5) -> pai2

        for i in range(crossover_point, 10): # crossover depois do corte
            child1_genes.set(i, not parent2_genes.get(i)) # filho1 = (6,9) -> pai2
            child2_genes.set(i, not parent1_genes.get(i)) # filho2 = (6,9) -> pai1

        child1 = Chromossome()
        child1.set_genes(child1_genes)
        print(f"1st child generated from crossover: {GeneticUtils.format_chromossome(child1)}")

        child2 = Chromossome()
        child2.set_genes(child2_genes)
        print(f"2nd child generated from crossover: {GeneticUtils.format_chromossome(child2)}")

        population.append(child1)
        population.append(child2)
        
    @staticmethod
    def mutation(population):
        prob = random.uniform(0, 1)

        if prob >= GeneticOperators.MUTATION_PROBABILITY:    # nao faz a mutacao
            return

        target = random.choice(population)

        mutation_point = random.randint(0, 9)
        print(f"Individual {target.to_string()} will mutate at point {mutation_point}")

        genes = target.get_genes()
        genes.flip(mutation_point)

        print(f"Individual {target.to_string()} mutated at point {mutation_point}")

    @staticmethod
    def elitism(population):            # retirando os dois piores cromossomos
        for _ in range(2):
            worst_individual = GeneticUtils.find_worst_chromossome(population)
            print(f"Removing worst individual from population: {GeneticUtils.format_chromossome(worst_individual)}")
            population.remove(worst_individual)

## Main

In [10]:
#import matplotlib.pyplot as plt

if __name__ == "__main__":
    population = []
    for _ in range(10):
        population.append(Chromossome())

    generation = 0
    population_score = Problem.g_average(population)

    generation_plot = []
    generation_plot.append(generation)

    population_score_plot = []
    population_score_plot.append(population_score)

    while generation < 100:
        parent1, parent2 = GeneticOperators.selection(population) # seleciona dois pais

        GeneticOperators.crossover(population, parent1, parent2)  # crossover para gerar dois filhos
        GeneticOperators.mutation(population)                     # chance de mutacao
        GeneticOperators.elitism(population)                      # remove os dois piores da populacao

        generation += 1
        population_score = Problem.g_average(population)

        generation_plot.append(generation)
        population_score_plot.append(population_score)

        print(f"Generation # {generation} -> Average population score = {population_score:.3f}\n")

    best_chromossome = GeneticUtils.find_best_chromossome(population)
    print(f"Best individual: {GeneticUtils.format_chromossome(best_chromossome)}")

    #plt.gca().set_xlabel('Generation')
    #plt.gca().set_ylabel('Average fitness')
    #plt.gca().set_title('Average fitness per generation')
    #plt.plot(generation_plot, population_score_plot)
    #plt.show()

1st parent chosen for crossover: G = [00110 11101], F = [6, -13], Score = 0.005
2nd parent chosen for crossover: G = [00110 01110], F = [6, 14], Score = 0.004
Crossover will happen at point 7
1st child generated from crossover: G = [00110 11110], F = [6, -14], Score = 0.004
2nd child generated from crossover: G = [00110 01101], F = [6, 13], Score = 0.005
Removing worst individual from population: G = [01101 11010], F = [13, -10], Score = 0.004
Removing worst individual from population: G = [01100 11011], F = [12, -11], Score = 0.004
Generation # 1 -> Average population score = 0.032

1st parent chosen for crossover: G = [01000 10010], F = [8, -2], Score = 0.014
2nd parent chosen for crossover: G = [11111 00001], F = [-15, 1], Score = 0.004
Crossover will happen at point 9
1st child generated from crossover: G = [01000 10011], F = [8, -3], Score = 0.014
2nd child generated from crossover: G = [11111 00000], F = [-15, 0], Score = 0.004
Removing worst individual from population: G = [1011

Generation # 85 -> Average population score = 1.000

1st parent chosen for crossover: G = [10000 10000], F = [0, 0], Score = 1.000
2nd parent chosen for crossover: G = [10000 10000], F = [0, 0], Score = 1.000
Crossover will happen at point 7
1st child generated from crossover: G = [10000 10000], F = [0, 0], Score = 1.000
2nd child generated from crossover: G = [10000 10000], F = [0, 0], Score = 1.000
Removing worst individual from population: G = [10000 10000], F = [0, 0], Score = 1.000
Removing worst individual from population: G = [10000 10000], F = [0, 0], Score = 1.000
Generation # 86 -> Average population score = 1.000

1st parent chosen for crossover: G = [10000 10000], F = [0, 0], Score = 1.000
2nd parent chosen for crossover: G = [10000 10000], F = [0, 0], Score = 1.000
Crossover will happen at point 1
1st child generated from crossover: G = [10000 10000], F = [0, 0], Score = 1.000
2nd child generated from crossover: G = [10000 10000], F = [0, 0], Score = 1.000
Individual G = [